# How to build a simple Agent LLM App with LangGraph
* Very basic tool-using Agent LLM App with memory.

Language models can't do anything on their own; they can only create text based on what you ask them. However, LangChain allows people to build agents—think of these as smart systems or helpers— that use language models to think and decide what to do next.

Here’s how it works:
1. **Use the Language Model as a Brain**: The agent uses the language model to figure out which actions it should take, based on the information it has or what you ask it to do.
2. **Taking Action**: After deciding, the agent then goes ahead and does those actions.
3. **Learning and Adjusting**: Once the actions are done, the results can be given back to the language model. This helps the model check if everything is complete or if it needs to do something else.

So, essentially, LangChain helps turn a language model from just a tool for writing and answering into a system that can act and react, almost like a very simple robot brain.

Here we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it calling the search tool, and have conversations with it.

## Concepts included
* [LangGraph](https://python.langchain.com/v0.2/docs/concepts/#langgraph).

LangGraph is a library created by the LangChain team for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows.

In [1]:
#!pip install python-dotenv

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install langchain

## Connect with an LLM and start a conversation with it

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain-openai

* For this project, we will use OpenAI's gpt-3.5-turbo

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

#### Track the operation in LangSmith
* [Open LangSmith here](smith.langchain.com)

## Agents
* Agents use LLMs as reasoning engines to determine which actions to take.

## Tool-using Agent
* For this basic agent we will use just one tool. In next advanced projects, you will learn how to use agents with several tools.
* **Our tool of choice will be Tavily** - a search engine. 
#### Tavily API Key
* You will need to add your Tavily API key in the .env file. Obtain a key by signing up on their [website](https://tavily.com/).

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [6]:
#pip install langgraph

#### Define tools

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [7]:
#pip install langchain-community

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
search.invoke("Who are the top stars of the 2024 Eurocup?")

[{'url': 'https://talksport.com/football/1755223/euro-2024-best-players-cristiano-ronaldo-harry-kane-jude-bellingham-kylian-mbappe/',
  'content': "Kane is just one of the names of serious talent that are set to star at Euro 2024 Credit: Getty. Gareth Southgate's side face stiff competition though as World Cup finalists France remain one of ..."},
 {'url': 'https://www.thescore.com/fran/news/2919037',
  'content': "Ranking the 24 best players at Euro 2024. Gordon Brunt, Anthony Lopopolo, Daniel Rouse. 4mo ago. Getty. We've already compiled a list of the most notable players to miss out on Euro 2024. Now ..."}]

In [7]:
tools = [search]

In [8]:
tools

[TavilySearchResults(max_results=2)]

* **In order to enable this model to do tool calling we use .bind_tools** to give the language model knowledge of these tools:

In [9]:
llm_with_tools = llm.bind_tools(tools)

## Create the agent
* We will be using LangGraph to construct the agent. 
* **Note that below we are passing in the origina chat model, not the llm_with_tools we built later**. That is because create_tool_calling_executor will call .bind_tools for us under the hood.

In [11]:
# from langgraph.prebuilt import chat_agent_executor

# agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools)

* In the latest version of langgraph, we have a most modern way to do this:

In [10]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

## Run the agent
* Let's first try it with .invoke():

In [11]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="Where is the soccer Eurocup 2024 played?")]})

response["messages"]

[HumanMessage(content='Where is the soccer Eurocup 2024 played?', id='cbae213a-cb4f-480d-9c63-f9e1228f1671'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tKXp2e1JEtb5OcwCUgyGXCqm', 'function': {'arguments': '{"query":"Soccer Eurocup 2024 location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 92, 'total_tokens': 117, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c164ff59-1db0-41f5-86a5-1fa63a39e6bf-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Soccer Eurocup 2024 location'}, 'id': 'call_tKXp2e1JEtb5OcwCUgyGXCqm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 92, 'output_tokens': 25, 'total_tokens': 117}),
 ToolM

* Now let's try it with .stream():

In [12]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="When and where will it be the 2024 Eurocup final match?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fGESTXu6UepD99IBsI7Atcgm', 'function': {'arguments': '{"query":"2024 Eurocup final match date and location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 96, 'total_tokens': 122, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ceb56c34-6c3b-474e-9104-bc56d71f254f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 Eurocup final match date and location'}, 'id': 'call_fGESTXu6UepD99IBsI7Atcgm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 96, 'output_tokens': 26, 'total_tokens': 122})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "htt

## Adding memory
* Adding memory in LangGraph is very similar to what we did with LangChain.

In [13]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

* Let's create our new agent with memory and set one thread_id so the agent can create a memory for each session as we did with our previous conversational RAG app:

In [14]:
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "001"}}

* Let's now try this new agent with .stream():

In [15]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who won the 2024 soccer Eurocup?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PEfM7HRMF1vv1gNWo5lzNP0B', 'function': {'arguments': '{"query":"2024 soccer Eurocup winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 91, 'total_tokens': 114, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-acadbdd4-6e1f-4fc5-9d54-dd1caba300e5-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 soccer Eurocup winner'}, 'id': 'call_PEfM7HRMF1vv1gNWo5lzNP0B', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 23, 'total_tokens': 114})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://en.wikipedia.org/wiki/UEFA_

In [16]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who were the top stars of that winner team?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pAsEfrjasFH8kqcsVTnkfDTh', 'function': {'arguments': '{"query":"top players of Spain national team Euro 2024"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 391, 'total_tokens': 418, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1b504ea3-48c9-47c5-b343-a27a17384388-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'top players of Spain national team Euro 2024'}, 'id': 'call_pAsEfrjasFH8kqcsVTnkfDTh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 391, 'output_tokens': 27, 'total_tokens': 418})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url"

* Let's change the thread_id and see what happens:

In [17]:
config = {"configurable": {"thread_id": "002"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="About what soccer team we were talking?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='You mentioned the soccer team "Manchester United" in our previous conversation. Would you like more information about them?', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 89, 'total_tokens': 112, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ca6a8749-bc43-4123-b89e-ff15f7aac98c-0', usage_metadata={'input_tokens': 89, 'output_tokens': 23, 'total_tokens': 112})]}}
----


* As you can see, when we changed the threat_id we started a different conversation memory, so our app does not remember the previous interaction and instead of giving us the right answer it decided to hallucinate.

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 001-simple-agent.py
* In terminal, make sure you are in the directory of the file and run:
    * python 001-simple-agent.py